In [1]:
from sklearn.preprocessing import LabelEncoder
import torch
import pandas as pd
import numpy as np
import pickle

In [2]:
BSC = pd.read_csv('./original_data/Base_station_basic_info.csv')
CLD = pd.read_csv('./original_data/Cell-level_data.csv')
ECD = pd.read_csv('./original_data/Energy_consumption_data.csv')

In [3]:
merged_df = CLD.merge(BSC, on=['BS', 'CellName'], how='inner')
fullmerged_df = merged_df.merge(ECD, on=['BS', 'Time'], how='left') # merge + wyjscie

In [4]:
unique_cells_per_base = fullmerged_df.groupby('BS')['CellName'].nunique()
one_cell_bases = unique_cells_per_base[unique_cells_per_base == 1].index
two_cell_bases = unique_cells_per_base[unique_cells_per_base == 2].index
four_cell_bases = unique_cells_per_base[unique_cells_per_base == 4].index

In [5]:
fullmerged_df_1 = fullmerged_df[fullmerged_df['BS'].isin(one_cell_bases)]
fullmerged_df_2 = fullmerged_df[fullmerged_df['BS'].isin(two_cell_bases)]
fullmerged_df_4 = fullmerged_df[fullmerged_df['BS'].isin(four_cell_bases)]

fullmerged_df_24 = fullmerged_df[fullmerged_df['BS'].isin(four_cell_bases) | fullmerged_df['BS'].isin(two_cell_bases)]

# print(len(fullmerged_df_2), len(fullmerged_df_24), end='\n\n')

In [6]:
condition = (fullmerged_df_24['CellName'] != 'Cell2') & (fullmerged_df_24['CellName'] != 'Cell3')
fullmerged_df_24 = fullmerged_df_24[condition]
fullmerged_df_2 = fullmerged_df_24

joined_df_2 = fullmerged_df_2.pivot_table(index=['Time', 'BS'], columns='CellName', values=['load', 'ESMode1', 'ESMode2', 'ESMode3',
       'ESMode4', 'ESMode5', 'ESMode6', 'Frequency', 'Bandwidth', 'TXpower'], aggfunc='sum', fill_value=999)

# dokleic energy i antennas
joined_df_2.columns = ['{}_{}'.format(col[1], col[0]) for col in joined_df_2.columns]
joined_df_2 = joined_df_2.sort_index(axis=1)
joined_df_2 = joined_df_2.reset_index()
joined_df_2 = joined_df_2.merge(fullmerged_df_2[['Time', 'BS', 'RUType', 'Mode', 'Antennas', 'Energy']], on=['Time', 'BS'], how='left')
joined_df_2 = joined_df_2.drop_duplicates()

In [7]:
joined_df_2['Time'] = pd.to_datetime(joined_df_2['Time'], format='%m/%d/%Y %H:%M')

joined_df_2['Year'] = joined_df_2['Time'].dt.year
joined_df_2['Day'] = joined_df_2['Time'].dt.day
joined_df_2['Hour'] = joined_df_2['Time'].dt.hour
joined_df_2['Month'] = joined_df_2['Time'].dt.month

joined_df_2 = joined_df_2.sort_values(by=['BS','Time'])
joined_df_2 = joined_df_2.reset_index()
# joined_df_2.to_csv('./joined_df_2.csv', index=False)

In [8]:
#ewentualne wypelnienie dziur predykcjami i oczywistymi strzalami
with open('./pickle/all_preds_loss_1_64.pkl', 'rb') as file:
    all_preds_dict = pickle.load(file)

joined_df_2.reset_index(inplace=True, drop=True)

for index, row in joined_df_2.iterrows():
    energy = row.Energy
    
    if np.isnan(energy):
        try:
            energy_pred = float(all_preds_dict[(row.BS, str(row.Time))])
            joined_df_2.loc[index, 'Energy'] = energy_pred
        except:
            print("gowno",row.BS, row.Time)


In [9]:
joined_df_2['Energy_1'] = joined_df_2['Energy'].shift(1, fill_value=0)
bs = None
past_energy_cell2_dict = {}
for index, row in joined_df_2.iterrows():
    time = row.Time
    if bs != row.BS:
        bs = row.BS
        joined_df_2.at[index, 'Energy_1'] = 0.0
    elif np.isnan(row.Energy_1):
        try:
            joined_df_2.at[index, 'Energy_1'] = joined_df_2.at[index-1, 'Energy_1']
        except:
            print('Mamy problem')
    past_energy_cell2_dict[(bs, time)] = [joined_df_2.at[index, 'Energy_1']]
    
# with open('./pickle/past_energy_cell2_dict.pkl', 'wb') as file:
#     pickle.dump(past_energy_cell2_dict, file)

In [10]:
big_dict = {}
for row in joined_df_2.itertuples(index=False):
    bs = row.BS
    bw = row.Cell1_Bandwidth
    es_mode1 = row.Cell1_ESMode1
    es_mode2 = row.Cell1_ESMode2
    es_mode3 = row.Cell1_ESMode3
    es_mode4 = row.Cell1_ESMode4
    es_mode5 = row.Cell1_ESMode5
    es_mode6 = row.Cell1_ESMode6
    fr = row.Cell1_Frequency
    txp = row.Cell1_TXpower
    load = row.Cell1_load
    hour = row.Hour
    if bw != 999:
        big_dict[(bs, hour)] = [bw, es_mode1, es_mode2, es_mode3, es_mode4, es_mode5, es_mode6, fr, txp, load]

with open('./pickle/big_dict.pkl', 'wb') as file:
    pickle.dump(big_dict, file)

In [11]:
# joined_df_2["Info"] = 0
joined_df_2.to_csv('./joined_df_2.csv', index=False)
for index, row in joined_df_2.iterrows():
    if row.Cell1_Bandwidth == 999:
        tmp_list = big_dict[(row.BS, row.Hour)]
        joined_df_2.at[index, 'Cell1_Bandwidth'] = tmp_list[0]
        joined_df_2.at[index, 'Cell1_ESMode1'] = tmp_list[1]
        joined_df_2.at[index, 'Cell1_ESMode2'] = tmp_list[2]
        joined_df_2.at[index, 'Cell1_ESMode3'] = tmp_list[3]
        joined_df_2.at[index, 'Cell1_ESMode4'] = tmp_list[4]
        joined_df_2.at[index, 'Cell1_ESMode5'] = tmp_list[5]
        joined_df_2.at[index, 'Cell1_ESMode6'] = tmp_list[6]
        joined_df_2.at[index, 'Cell1_Frequency'] = tmp_list[7]
        joined_df_2.at[index, 'Cell1_TXpower'] = tmp_list[8]
        joined_df_2.at[index, 'Cell1_load'] = tmp_list[9]
        # joined_df_2.at[index, 'Info'] = 1
# print(joined_df_2["Info"].value_counts())

In [12]:
#zmiana stringa np. RUType na liczby
label_encoders = {}
object_cols = ['RUType', 'Mode'] 

for col in object_cols:
    le = LabelEncoder()
    joined_df_2[col] = le.fit_transform(joined_df_2[col])
    label_encoders[col] = le
joined_df_2['RUType'].value_counts()

RUType
0    25090
3     1259
2      139
4      139
1       48
5       24
Name: count, dtype: int64

In [13]:
# joined_df_2.to_csv('./joined_df_2.csv', index=False)
Encoded_rutype = torch.nn.functional.one_hot(torch.tensor(joined_df_2['RUType'].values).long(), len(joined_df_2['RUType'].value_counts()))
Encoded_rutype = pd.DataFrame(Encoded_rutype, columns=[f"RUType{i}" for i in range(len(joined_df_2['RUType'].value_counts()))])
Encoded_rutype.value_counts()

RUType0  RUType1  RUType2  RUType3  RUType4  RUType5
1        0        0        0        0        0          25090
0        0        0        1        0        0           1259
                           0        1        0            139
                  1        0        0        0            139
         1        0        0        0        0             48
         0        0        0        0        1             24
Name: count, dtype: int64

In [14]:
joined_df_2_numpy = joined_df_2.to_numpy()
Encoded_rutype_numpy = Encoded_rutype.to_numpy()
chuj = joined_df_2.columns
joined_df_2 = np.concatenate((joined_df_2_numpy,Encoded_rutype_numpy), axis=1)
joined_df_2 = pd.DataFrame(joined_df_2, columns = chuj.append(Encoded_rutype.columns))

# joined_df_2 = joined_df_2.drop(["BS", "Year", "Month", "RUType", "Time", "index"], axis=1)
joined_df_2 = joined_df_2.drop(["Year", "Month", "RUType", "index"], axis=1)
# joined_df_2 = joined_df_2.drop(["BS", "Time"], axis=1)
# joined_df_2 = joined_df_2.dropna(subset=['Energy'])
cols = [col for col in joined_df_2 if col != 'Energy'] + ['Energy']
joined_df_2 = joined_df_2[cols]

In [15]:
joined_df_2.to_csv('./prepared_data/TwoOrFour_Cell_merged_with_preds.csv', index=False)
print(joined_df_2.columns)

Index(['Time', 'BS', 'Cell0_Bandwidth', 'Cell0_ESMode1', 'Cell0_ESMode2',
       'Cell0_ESMode3', 'Cell0_ESMode4', 'Cell0_ESMode5', 'Cell0_ESMode6',
       'Cell0_Frequency', 'Cell0_TXpower', 'Cell0_load', 'Cell1_Bandwidth',
       'Cell1_ESMode1', 'Cell1_ESMode2', 'Cell1_ESMode3', 'Cell1_ESMode4',
       'Cell1_ESMode5', 'Cell1_ESMode6', 'Cell1_Frequency', 'Cell1_TXpower',
       'Cell1_load', 'Mode', 'Antennas', 'Day', 'Hour', 'Energy_1', 'RUType0',
       'RUType1', 'RUType2', 'RUType3', 'RUType4', 'RUType5', 'Energy'],
      dtype='object')
